# Fine-tune BLIP using Hugging Face `transformers` and `datasets` 🤗

This tutorial is largely based from the [GiT tutorial](https://colab.research.google.com/drive/1HLxgrG7xZJ9FvXckNG61J72FkyrbqKAA?usp=sharing) on how to fine-tune GiT on a custom image captioning dataset. Here we will use a dummy dataset of [football players](https://huggingface.co/datasets/ybelkada/football-dataset) ⚽ that is uploaded on the Hub. The images have been manually selected together with the captions.
Check the 🤗 [documentation](https://huggingface.co/docs/datasets/image_dataset) on how to create and upload your own image-text dataset.

## Set-up environment

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main

In [ ]:
!pip install -q datasets

## Load the image captioning dataset

Let's load the image captioning dataset, you just need few lines of code for that.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ybelkada/football-dataset", split="train")

Let's retrieve the caption of the first example:

In [ ]:
dataset[0]["text"]

And the corresponding image

In [ ]:
dataset[0]["image"]

## Create PyTorch Dataset

The lines below are entirely copied from the original notebook!

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

## Load model and processor

In [ ]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Now that we have loaded the processor, let's load the dataset and the dataloader:

In [ ]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)

## Train the model

Let's train the model! Run the simply the cell below for training the model

In [ ]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(50):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

/tmp/ipython-input-4169383787.py:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=use_amp)
/tmp/ipython-input-4169383787.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):


Epoch 1/5 | step 50 | train_loss(avg50)=2.1184
Epoch 1/5 | step 100 | train_loss(avg50)=2.1226
Epoch 1/5 | step 150 | train_loss(avg50)=2.0402
Epoch 1/5 | step 200 | train_loss(avg50)=2.1251
Epoch 1/5 | step 250 | train_loss(avg50)=2.0582
Epoch 1/5 | step 300 | train_loss(avg50)=2.0800
Epoch 1/5 | step 350 | train_loss(avg50)=2.0489
Epoch 1/5 | step 400 | train_loss(avg50)=2.0886
Epoch 1/5 | step 450 | train_loss(avg50)=2.0177
Epoch 1/5 | step 500 | train_loss(avg50)=2.1440


[VAL] step 500 | BLEU=0.2432 | ROUGE-L=0.5098
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black and white dog is jumping over a log.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a person in a blue jacket is hanging from a rope.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a man and a child are sitting in a yellow kayak.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little boy in blue life jackets are rowing a

[VAL] step 1000 | BLEU=0.2432 | ROUGE-L=0.5110
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black dog is jumping over a log.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a man is rock climbing.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a man in a life vest is paddling a yellow kayak with a child in a life vest.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little boy in blue life jackets are rowing a yellow

[VAL] step 1500 | BLEU=0.2310 | ROUGE-L=0.4993
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black and white dog is playing with a large log.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a man in a blue shirt and black pants is climbing up a snowy mountain.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a boy in a yellow kayak paddles another boy in a yellow kayak.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a li

...

[VAL] step 11000 | BLEU=0.2391 | ROUGE-L=0.4982
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black dog is jumping over a fallen tree in the forest.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a man in a blue jacket is climbing up a snowy hill.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a woman and a child are sitting in a yellow canoe on the water.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little boy in

[VAL] step 11500 | BLEU=0.2258 | ROUGE-L=0.4945
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black dog is jumping over a fallen tree in the grass.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a man in a blue jacket is climbing up a snowy mountain.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a man and a child in a life jacket in a yellow kayak on blue water.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little

[VAL] step 12000 | BLEU=0.2309 | ROUGE-L=0.4948
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black dog jumps over a log.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a person in a blue jacket is hanging upside down in the snow.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a man holds a paddle in a yellow kayak.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little boy in blue life jackets are rowing a yellow ca

[VAL] step 12500 | BLEU=0.2058 | ROUGE-L=0.4776
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black dog is jumping over a fallen tree.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a man in a blue jacket climbs up a snowy hill.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a man and a toddler in a yellow kayak ride through the water.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man and a little boy in blue life jackets ar

[VAL] step 13000 | BLEU=0.2285 | ROUGE-L=0.4873
  sample image: 1015584366_dfcec3c85a.jpg
  pred: a black and white dog is jumping over a large piece of wood.
  refs: ['A black dog leaps over a log .', 'A grey dog is leaping over a fallen tree .', 'A large black dog leaps a fallen log .', 'A mottled black and grey dog in a blue collar jumping over a fallen tree .', 'The black dog jumped the tree stump .']
  ---
  sample image: 102455176_5f8ead62d5.jpg
  pred: a person wearing a blue jacket is skiing down a snowy mountain.
  refs: ['A man uses ice picks and crampons to scale ice .', 'an ice climber in a blue jacket and black pants is scaling a frozen ice wall .', 'An ice climber scaling a frozen waterfall .', 'A person in blue and red ice climbing with two picks .', 'Climber climbing an ice wall']
  ---
  sample image: 1028205764_7e8df9a2ea.jpg
  pred: a young boy and a young girl are in a yellow kayak on calm river.
  refs: ['A man and a baby are in a yellow kayak on water .', 'A man a

## Inference

Let's check the results on our train dataset

In [ ]:
# load image
example = dataset[0]
image = example["image"]
image

In [ ]:
# prepare image for the model
inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values

generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)

## Load from the Hub

Once trained you can push the model and processor on the Hub to use them later.
Meanwhile you can play with the model that we have fine-tuned!

In [ ]:
from transformers import BlipForConditionalGeneration, AutoProcessor

model = BlipForConditionalGeneration.from_pretrained("ybelkada/blip-image-captioning-base-football-finetuned").to(device)
processor = AutoProcessor.from_pretrained("ybelkada/blip-image-captioning-base-football-finetuned")

Let's check the results on our train dataset!

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(18, 14))

# prepare image for the model
for i, example in enumerate(dataset):
  image = example["image"]
  inputs = processor(images=image, return_tensors="pt").to(device)
  pixel_values = inputs.pixel_values

  generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
  generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  fig.add_subplot(2, 3, i+1)
  plt.imshow(image)
  plt.axis("off")
  plt.title(f"Generated caption: {generated_caption}")